This script can be used for converting data from DICOM to NIFTI. You can choose `data_type` from `['FLAIR', 'T1w', 'T1wCE', 'T2w']`

In [ ]:
"""Script for converting DICOM to nifti"""

import concurrent.futures
import logging
import os
import warnings
from functools import partial
from pathlib import Path
from typing import Union

import SimpleITK as sitk
from tqdm import tqdm

logger = logging.getLogger(__name__)


def convert_dicom_folder(
    dicom_dir: str,
    nifti_dir: str,
    data_type: str,
    max_workers: int
):
    dicom_dir = Path(dicom_dir)
    nifti_dir = Path(nifti_dir)
    
    nifti_dir.mkdir(exist_ok=True)

    if data_type not in ['FLAIR', 'T1w', 'T1wCE', 'T2w']:
        raise ValueError(f'Bad data type: {data_type}!')

    pattern = os.path.join('**', data_type)
    folders_to_convert = list(dicom_dir.rglob(pattern=pattern))

    convert_dicom_partial = partial(
        convert_dicom_file_and_fix_align, output_folder=nifti_dir, compression=True
    )

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(
            tqdm(
                executor.map(convert_dicom_partial, folders_to_convert),
                total=len(folders_to_convert),
                disable=False,
                postfix='Converting DICOM to nifti...',
            )
        )


def convert_dicom_file_and_fix_align(
    dicom_directory: Union[str, Path],
    output_folder: Union[str, Path],
    compression: bool = True,
) -> None:
    dicom_directory = Path(dicom_directory).expanduser()
    output_folder = Path(output_folder).expanduser()

    image = load_dicom(dicom_directory=dicom_directory)

    res_file_path = get_nii_filepath(
        dicom_directory=dicom_directory,
        output_folder=output_folder,
        compression=compression,
    )

    sitk.WriteImage(
        image=image, fileName=str(res_file_path), useCompression=compression
    )


def load_dicom(dicom_directory: Union[str, Path]) -> sitk.Image:
    sitk.ProcessObject_SetGlobalWarningDisplay(False)

    series_ids = sitk.ImageSeriesReader.GetGDCMSeriesIDs(directory=str(dicom_directory))
    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(
        str(dicom_directory), series_ids[0]
    )
    series_reader = sitk.ImageSeriesReader()
    series_reader.SetFileNames(series_file_names)
    series_reader.LoadPrivateTagsOn()
    image = series_reader.Execute()

    return image


def get_nii_filepath(
    dicom_directory: Path,
    output_folder: Path,
    compression: bool = True,
) -> Path:
    sequence_type = str(dicom_directory).split(os.sep)[-1]
    sequence_id = str(dicom_directory).split(os.sep)[-2]

    res_file_name = f'{sequence_id}_{sequence_type}.nii'
    if compression:
        res_file_name += '.gz'

    res_file_path = output_folder.joinpath(res_file_name)

    return res_file_path

In [ ]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore')

    convert_dicom_folder(
        dicom_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train',
        nifti_dir='./nifti_result',
        data_type='FLAIR',
        max_workers=8
    )